In [14]:
# CS504 
# Team 1 
# Data https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3/about_data 

In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_32006/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [17]:
import findspark
findspark.init()

In [18]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession 
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import col, to_date
from pyspark.sql.functions import col, unix_timestamp, round
from pyspark.sql.functions import col, datediff, hour, dayofweek

sc = SparkContext.getOrCreate()
# spark = SparkSession.builder.getOrCreate()
# print(sc.version)
# print(spark.version)

# from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YourApp") \
    .config("spark.executor.memory", "15G") \
    .getOrCreate()

# read csv file with our defined schema into Spark DataFrame, and use "," delimiter
df = spark.read.csv(
    "/home/pk/Downloads/fire.csv", 
    header=True, sep=",")
#show 3 rows of our DataFrame
df.show(3)

+-----------+-------+---------------+----------------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+-------------+-------------------+---------+------------+----+-----------------+--------+--------------+--------+--------------------+----------------+---------+------------------------------+------------------------+-------------------+------------------------------------+-------------+--------------------+--------------------+--------------------+----------------------+
|Call Number|Unit ID|Incident Number|       Call Type| Call Date|Watch Date|       Received DtTm|          Entry DtTm|       Dispatch DtTm|       Response DtTm|       On Scene DtTm|      Transport DtTm|       Hospital DtTm|Call Final Disposition|      Available DtTm|             Address|         City|Zipcode of Incident|Battalion|Stati

In [19]:
#count number of rows of our dataFrame
num_rows = df.count()
print("number of rows: ", num_rows)

number of rows:  1992749


In [20]:
df.printSchema()

root
 |-- Call Number: string (nullable = true)
 |-- Unit ID: string (nullable = true)
 |-- Incident Number: string (nullable = true)
 |-- Call Type: string (nullable = true)
 |-- Call Date: string (nullable = true)
 |-- Watch Date: string (nullable = true)
 |-- Received DtTm: string (nullable = true)
 |-- Entry DtTm: string (nullable = true)
 |-- Dispatch DtTm: string (nullable = true)
 |-- Response DtTm: string (nullable = true)
 |-- On Scene DtTm: string (nullable = true)
 |-- Transport DtTm: string (nullable = true)
 |-- Hospital DtTm: string (nullable = true)
 |-- Call Final Disposition: string (nullable = true)
 |-- Available DtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode of Incident: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- Station Area: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- Original Priority: string (nullable = true)
 |-- Priority: string (nullable = 

In [21]:
# Rename columns to remove spaces
columns = df.columns
renamed_columns = [col.replace(" ", "_") for col in columns]
df = df.toDF(*renamed_columns)

# Convert date fields to datetime format
date_fields = ["Call_Date", "Watch_Date"]  
    
for field in date_fields:
    df = df.withColumn(field, to_date(df[field], "MM/dd/yyyy"))
    
    
# Convert date fields to datetime format
date_time_fields = ["Received_DtTm", "Entry_DtTm", "Dispatch_DtTm", "Response_DtTm", 
               "On_Scene_DtTm", "Transport_DtTm", "Hospital_DtTm", "Available_DtTm"]

# for field in date_fields:
#     df = df.withColumn(field, df[field].cast("timestamp"))
    
    
for field in date_time_fields:
    df = df.withColumn(field, to_timestamp(df[field], "MM/dd/yyyy hh:mm:ss a"))
    
df.printSchema()

root
 |-- Call_Number: string (nullable = true)
 |-- Unit_ID: string (nullable = true)
 |-- Incident_Number: string (nullable = true)
 |-- Call_Type: string (nullable = true)
 |-- Call_Date: date (nullable = true)
 |-- Watch_Date: date (nullable = true)
 |-- Received_DtTm: timestamp (nullable = true)
 |-- Entry_DtTm: timestamp (nullable = true)
 |-- Dispatch_DtTm: timestamp (nullable = true)
 |-- Response_DtTm: timestamp (nullable = true)
 |-- On_Scene_DtTm: timestamp (nullable = true)
 |-- Transport_DtTm: timestamp (nullable = true)
 |-- Hospital_DtTm: timestamp (nullable = true)
 |-- Call_Final_Disposition: string (nullable = true)
 |-- Available_DtTm: timestamp (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode_of_Incident: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- Station_Area: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- Original_Priority: string (nullable = true)
 |-- Priority:

In [22]:
# Filter out data (2018-2023 only)
df = df.filter(col("Call_Date") <= "2023-12-31")
#count number of rows of our dataFrame
num_rows = df.count()
print("number of rows: ", num_rows)

number of rows:  1910643


In [23]:
# Calculate Response_Time
# df = df.withColumn("Response_Time", (unix_timestamp("On_Scene_DtTm") - unix_timestamp("Received_DtTm")) / 60)
df = df.withColumn("Response_Time", round((unix_timestamp("On_Scene_DtTm") - unix_timestamp("Received_DtTm")) / 60, 2))
df = df.withColumn("Received_Hour", hour("Received_DtTm")).withColumn("Received_DayOfWeek", dayofweek("Received_DtTm"))

# Count the number of null values in the On_Scene_DtTm column
null_count = df.filter(col("On_Scene_DtTm").isNull()).count()

print("Number of null values in On_Scene_DtTm:", null_count)

df.printSchema()


Number of null values in On_Scene_DtTm: 375611
root
 |-- Call_Number: string (nullable = true)
 |-- Unit_ID: string (nullable = true)
 |-- Incident_Number: string (nullable = true)
 |-- Call_Type: string (nullable = true)
 |-- Call_Date: date (nullable = true)
 |-- Watch_Date: date (nullable = true)
 |-- Received_DtTm: timestamp (nullable = true)
 |-- Entry_DtTm: timestamp (nullable = true)
 |-- Dispatch_DtTm: timestamp (nullable = true)
 |-- Response_DtTm: timestamp (nullable = true)
 |-- On_Scene_DtTm: timestamp (nullable = true)
 |-- Transport_DtTm: timestamp (nullable = true)
 |-- Hospital_DtTm: timestamp (nullable = true)
 |-- Call_Final_Disposition: string (nullable = true)
 |-- Available_DtTm: timestamp (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode_of_Incident: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- Station_Area: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- Original_Pr

In [24]:
categorical_cols = [
    "Call_Type", "City","Zipcode_of_Incident", "Battalion", "Station_Area", "Original_Priority",
    "Priority", "Final_Priority", "ALS_Unit", "Call_Type_Group",
    "Unit_Type", "Unit_sequence_in_call_dispatch", "Fire_Prevention_District",
    "Supervisor_District", "Neighborhooods_-_Analysis_Boundaries","Received_Hour", "Received_DayOfWeek"
]


from pyspark.sql import functions as F

unique_counts_df = df.select(*(F.countDistinct(col).alias(col) for col in categorical_cols))
unique_counts_dict = unique_counts_df.first().asDict()

for key, val in unique_counts_dict.items():
    print(f"{key}: {val}")




Call_Type: 32
City: 8
Zipcode_of_Incident: 27
Battalion: 17
Station_Area: 46
Original_Priority: 9
Priority: 9
Final_Priority: 2
ALS_Unit: 2
Call_Type_Group: 4
Unit_Type: 12
Unit_sequence_in_call_dispatch: 104
Fire_Prevention_District: 11
Supervisor_District: 12
Neighborhooods_-_Analysis_Boundaries: 42
Received_Hour: 24
Received_DayOfWeek: 7


In [25]:
#####################################

In [26]:
# Predicting Call Type Group Improved Model

from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("Call_Type_Group_Prediction") \
    .getOrCreate()

# Handle null or NaN values in Call_Type_Group by dropping rows with null labels
df = df.dropna(subset=["Call_Type_Group"])

# Convert Call_Type_Group from string to numeric using StringIndexer
label_indexer = StringIndexer(inputCol="Call_Type_Group", outputCol="label")
df = label_indexer.fit(df).transform(df)

selected_features = [
    "Call_Type", "City", "Zipcode_of_Incident", "Battalion", "Station_Area",
    "Original_Priority", "Priority", "Final_Priority", "ALS_Unit", "Number_of_Alarms",
    "Unit_Type", "Unit_sequence_in_call_dispatch", "Fire_Prevention_District",
    "Supervisor_District", "Neighborhooods_-_Analysis_Boundaries", "Received_Hour", "Received_DayOfWeek"
]

data = df.select(selected_features + ["label"])

# Handle categorical features by indexing and one-hot encoding them
categorical_cols = [
    "Call_Type", "City", "Zipcode_of_Incident", "Battalion", "Station_Area", "Original_Priority",
    "Priority", "Final_Priority", "ALS_Unit", "Number_of_Alarms",
    "Unit_Type", "Unit_sequence_in_call_dispatch", "Fire_Prevention_District",
    "Supervisor_District", "Neighborhooods_-_Analysis_Boundaries", "Received_Hour", "Received_DayOfWeek"
]

# Index categorical columns with handling of invalid input set to "keep"
indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep").fit(data)
    for col in categorical_cols
]

# One-hot encode indexed categorical columns
encoders = [
    OneHotEncoder(inputCol=col+"_index", outputCol=col+"_onehot")
    for col in categorical_cols
]

pipeline = Pipeline(stages=indexers + encoders)
data_encoded = pipeline.fit(data).transform(data)

# Prepare data for modeling
assembler = VectorAssembler(inputCols=[col + "_onehot" for col in categorical_cols], outputCol="features")
data_final = assembler.transform(data_encoded).select("features", "label")

# Split data into training and test sets (80% train, 20% test)
train_data, test_data = data_final.randomSplit([0.8, 0.2], seed=42)

# Define hyperparameters for RandomForestClassifier
rf = RandomForestClassifier(featuresCol="features", labelCol="label")
param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20, 30]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .build()

# Set up 5-fold cross-validation
cv = CrossValidator(estimator=rf, estimatorParamMaps=param_grid, evaluator=MulticlassClassificationEvaluator(), numFolds=5)

# Train the model using cross-validation
cv_model = cv.fit(train_data)

# Make predictions on test data
predictions = cv_model.transform(test_data)

# Evaluate model performance
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

# cv_model.bestModel.write().overwrite().save("path_to_save_model")


24/04/21 13:23:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/04/21 13:25:06 WARN MemoryStore: Not enough space to cache rdd_213_3 in memory! (computed 13.7 MiB so far)
24/04/21 13:25:06 WARN BlockManager: Persisting block rdd_213_3 to disk instead.
24/04/21 13:25:06 WARN MemoryStore: Not enough space to cache rdd_213_0 in memory! (computed 8.4 MiB so far)
24/04/21 13:25:06 WARN BlockManager: Persisting block rdd_213_0 to disk instead.
24/04/21 13:25:06 WARN MemoryStore: Not enough space to cache rdd_213_2 in memory! (computed 13.7 MiB so far)
24/04/21 13:25:06 WARN BlockManager: Persisting block rdd_213_2 to disk instead.
24/04/21 13:25:06 WARN MemoryStore: Not enough space to cache rdd_213_4 in memory! (computed 8.4 MiB so far)
24/04/21 13:25:06 WARN BlockManager: Persisting block rdd_213_4 to disk instead.
24/04/21 13:25:06 WARN MemoryStore: Not enough space to cache rdd_213_7 in memory! (computed 13.7 MiB so far)
24/04/21

24/04/21 13:26:11 WARN MemoryStore: Not enough space to cache rdd_271_2 in memory! (computed 166.4 MiB so far)
24/04/21 13:26:17 WARN MemoryStore: Not enough space to cache rdd_271_0 in memory! (computed 46.5 MiB so far)
24/04/21 13:26:17 WARN MemoryStore: Not enough space to cache rdd_271_1 in memory! (computed 46.5 MiB so far)
24/04/21 13:26:17 WARN MemoryStore: Not enough space to cache rdd_271_6 in memory! (computed 46.5 MiB so far)
24/04/21 13:26:17 WARN MemoryStore: Not enough space to cache rdd_271_3 in memory! (computed 46.5 MiB so far)
24/04/21 13:26:17 WARN MemoryStore: Not enough space to cache rdd_271_5 in memory! (computed 46.5 MiB so far)
24/04/21 13:26:18 WARN MemoryStore: Not enough space to cache rdd_271_4 in memory! (computed 46.5 MiB so far)
24/04/21 13:26:18 WARN MemoryStore: Not enough space to cache rdd_271_2 in memory! (computed 70.2 MiB so far)
24/04/21 13:26:18 WARN MemoryStore: Not enough space to cache rdd_271_7 in memory! (computed 70.2 MiB so far)
24/04/21 

24/04/21 13:28:29 WARN MemoryStore: Not enough space to cache rdd_339_7 in memory! (computed 46.5 MiB so far)
24/04/21 13:28:29 WARN MemoryStore: Not enough space to cache rdd_339_1 in memory! (computed 46.5 MiB so far)
24/04/21 13:28:29 WARN MemoryStore: Not enough space to cache rdd_339_2 in memory! (computed 46.5 MiB so far)
24/04/21 13:28:29 WARN MemoryStore: Not enough space to cache rdd_339_6 in memory! (computed 46.5 MiB so far)
24/04/21 13:28:29 WARN MemoryStore: Not enough space to cache rdd_339_3 in memory! (computed 46.5 MiB so far)
24/04/21 13:28:29 WARN MemoryStore: Not enough space to cache rdd_339_5 in memory! (computed 46.5 MiB so far)
24/04/21 13:28:29 WARN MemoryStore: Not enough space to cache rdd_339_0 in memory! (computed 70.2 MiB so far)
24/04/21 13:28:29 WARN MemoryStore: Not enough space to cache rdd_339_4 in memory! (computed 70.2 MiB so far)
24/04/21 13:28:36 WARN DAGScheduler: Broadcasting large task binary with size 1167.3 KiB
24/04/21 13:28:36 WARN MemorySt

24/04/21 13:29:38 WARN MemoryStore: Not enough space to cache rdd_422_0 in memory! (computed 28.3 MiB so far)
24/04/21 13:29:38 WARN MemoryStore: Not enough space to cache rdd_422_1 in memory! (computed 28.3 MiB so far)
24/04/21 13:29:38 WARN MemoryStore: Not enough space to cache rdd_422_3 in memory! (computed 18.2 MiB so far)
24/04/21 13:29:38 WARN MemoryStore: Not enough space to cache rdd_422_7 in memory! (computed 28.3 MiB so far)
24/04/21 13:29:38 WARN MemoryStore: Not enough space to cache rdd_422_4 in memory! (computed 143.2 MiB so far)
24/04/21 13:29:38 WARN MemoryStore: Not enough space to cache rdd_422_5 in memory! (computed 18.2 MiB so far)
24/04/21 13:29:38 WARN MemoryStore: Not enough space to cache rdd_422_6 in memory! (computed 28.3 MiB so far)
24/04/21 13:29:38 WARN MemoryStore: Not enough space to cache rdd_422_2 in memory! (computed 143.2 MiB so far)
24/04/21 13:29:50 WARN MemoryStore: Not enough space to cache rdd_422_1 in memory! (computed 42.4 MiB so far)
24/04/21

24/04/21 13:31:25 WARN MemoryStore: Not enough space to cache rdd_475_2 in memory! (computed 42.4 MiB so far)
24/04/21 13:31:25 WARN MemoryStore: Not enough space to cache rdd_475_0 in memory! (computed 42.4 MiB so far)
24/04/21 13:31:25 WARN MemoryStore: Not enough space to cache rdd_475_5 in memory! (computed 42.4 MiB so far)
24/04/21 13:31:25 WARN MemoryStore: Not enough space to cache rdd_475_3 in memory! (computed 42.4 MiB so far)
24/04/21 13:31:25 WARN MemoryStore: Not enough space to cache rdd_475_4 in memory! (computed 63.6 MiB so far)
24/04/21 13:31:25 WARN MemoryStore: Not enough space to cache rdd_475_7 in memory! (computed 63.6 MiB so far)
24/04/21 13:31:25 WARN MemoryStore: Not enough space to cache rdd_475_6 in memory! (computed 63.6 MiB so far)
24/04/21 13:31:25 WARN MemoryStore: Not enough space to cache rdd_475_1 in memory! (computed 63.6 MiB so far)
24/04/21 13:31:38 WARN MemoryStore: Not enough space to cache rdd_475_2 in memory! (computed 42.4 MiB so far)
24/04/21 1

...

24/04/21 15:13:14 WARN MemoryStore: Not enough space to cache rdd_3235_0 in memory! (computed 65.2 MiB so far)
24/04/21 15:13:14 WARN BlockManager: Persisting block rdd_3235_0 to disk instead.
24/04/21 15:13:18 WARN MemoryStore: Not enough space to cache rdd_3235_5 in memory! (computed 12.3 MiB so far)
24/04/21 15:13:18 WARN MemoryStore: Not enough space to cache rdd_3235_7 in memory! (computed 43.4 MiB so far)
24/04/21 15:13:18 WARN MemoryStore: Not enough space to cache rdd_3235_0 in memory! (computed 28.9 MiB so far)
24/04/21 15:13:18 WARN MemoryStore: Not enough space to cache rdd_3235_1 in memory! (computed 28.9 MiB so far)
24/04/21 15:13:18 WARN MemoryStore: Not enough space to cache rdd_3235_3 in memory! (computed 18.6 MiB so far)
24/04/21 15:13:18 WARN MemoryStore: Not enough space to cache rdd_3235_6 in memory! (computed 8.2 MiB so far)
24/04/21 15:13:19 WARN MemoryStore: Not enough space to cache rdd_3235_4 in memory! (computed 146.7 MiB so far)
24/04/21 15:13:19 WARN MemoryS

24/04/21 15:16:08 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
24/04/21 15:16:08 WARN MemoryStore: Not enough space to cache rdd_3235_3 in memory! (computed 43.4 MiB so far)
24/04/21 15:16:08 WARN MemoryStore: Not enough space to cache rdd_3235_4 in memory! (computed 43.4 MiB so far)
24/04/21 15:16:08 WARN MemoryStore: Not enough space to cache rdd_3235_7 in memory! (computed 43.4 MiB so far)
24/04/21 15:16:08 WARN MemoryStore: Not enough space to cache rdd_3235_2 in memory! (computed 43.4 MiB so far)
24/04/21 15:16:08 WARN MemoryStore: Not enough space to cache rdd_3235_5 in memory! (computed 43.4 MiB so far)
24/04/21 15:16:08 WARN MemoryStore: Not enough space to cache rdd_3235_6 in memory! (computed 65.2 MiB so far)
24/04/21 15:16:08 WARN MemoryStore: Not enough space to cache rdd_3235_1 in memory! (computed 65.2 MiB so far)
24/04/21 15:16:09 WARN MemoryStore: Not enough space to cache rdd_3235_0 in memory! (computed 65.2 MiB so far)
24/04/21 15:16:30 WARN DAG

24/04/21 15:19:32 WARN MemoryStore: Not enough space to cache rdd_3317_7 in memory! (computed 232.0 MiB so far)
24/04/21 15:19:32 WARN MemoryStore: Not enough space to cache rdd_3317_5 in memory! (computed 28.9 MiB so far)
24/04/21 15:19:32 WARN MemoryStore: Not enough space to cache rdd_3317_6 in memory! (computed 5.4 MiB so far)
24/04/21 15:19:53 WARN MemoryStore: Not enough space to cache rdd_3317_2 in memory! (computed 43.4 MiB so far)
24/04/21 15:19:53 WARN MemoryStore: Not enough space to cache rdd_3317_5 in memory! (computed 43.4 MiB so far)
24/04/21 15:19:53 WARN MemoryStore: Not enough space to cache rdd_3317_6 in memory! (computed 43.4 MiB so far)
24/04/21 15:19:53 WARN MemoryStore: Not enough space to cache rdd_3317_0 in memory! (computed 43.4 MiB so far)
24/04/21 15:19:53 WARN MemoryStore: Not enough space to cache rdd_3317_7 in memory! (computed 43.4 MiB so far)
24/04/21 15:19:54 WARN MemoryStore: Not enough space to cache rdd_3317_3 in memory! (computed 65.2 MiB so far)
2

24/04/21 15:23:04 WARN MemoryStore: Not enough space to cache rdd_3317_4 in memory! (computed 65.2 MiB so far)
24/04/21 15:23:04 WARN MemoryStore: Not enough space to cache rdd_3317_5 in memory! (computed 65.2 MiB so far)
24/04/21 15:23:05 WARN MemoryStore: Not enough space to cache rdd_3317_2 in memory! (computed 65.2 MiB so far)
24/04/21 15:23:31 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
24/04/21 15:23:31 WARN MemoryStore: Not enough space to cache rdd_3317_5 in memory! (computed 43.4 MiB so far)
24/04/21 15:23:31 WARN MemoryStore: Not enough space to cache rdd_3317_0 in memory! (computed 43.4 MiB so far)
24/04/21 15:23:31 WARN MemoryStore: Not enough space to cache rdd_3317_2 in memory! (computed 43.4 MiB so far)
24/04/21 15:23:31 WARN MemoryStore: Not enough space to cache rdd_3317_1 in memory! (computed 43.4 MiB so far)
24/04/21 15:23:31 WARN MemoryStore: Not enough space to cache rdd_3317_3 in memory! (computed 43.4 MiB so far)
24/04/21 15:23:31 WARN Mem

Accuracy: 0.8725875455553795


In [14]:
# Predicting Call Type Group Initial Model

from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("Call_Type_Group_Prediction") \
    .getOrCreate()

# Handle null or NaN values in Call_Type_Group by dropping rows with null labels
df = df.dropna(subset=["Call_Type_Group"])

# Convert Call_Type_Group from string to numeric using StringIndexer
label_indexer = StringIndexer(inputCol="Call_Type_Group", outputCol="label")
df = label_indexer.fit(df).transform(df)

selected_features = [
    "Call_Type", "City", "Zipcode_of_Incident", "Battalion", "Station_Area",
    "Original_Priority", "Priority", "Final_Priority", "ALS_Unit", "Number_of_Alarms",
    "Unit_Type", "Unit_sequence_in_call_dispatch", "Fire_Prevention_District",
    "Supervisor_District", "Neighborhooods_-_Analysis_Boundaries", "Received_Hour", "Received_DayOfWeek"
]

data = df.select(selected_features + ["label"])

# Handle categorical features by indexing and one-hot encoding them
categorical_cols = [
    "Call_Type", "City", "Zipcode_of_Incident", "Battalion", "Station_Area", "Original_Priority",
    "Priority", "Final_Priority", "ALS_Unit", "Number_of_Alarms",
    "Unit_Type", "Unit_sequence_in_call_dispatch", "Fire_Prevention_District",
    "Supervisor_District", "Neighborhooods_-_Analysis_Boundaries", "Received_Hour", "Received_DayOfWeek"
]

# Index categorical columns with handling of invalid input set to "keep"
indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep").fit(data)
    for col in categorical_cols
]

# One-hot encode indexed categorical columns
encoders = [
    OneHotEncoder(inputCol=col+"_index", outputCol=col+"_onehot")
    for col in categorical_cols
]

pipeline = Pipeline(stages=indexers + encoders)
data_encoded = pipeline.fit(data).transform(data)

# Prepare data for modeling
assembler = VectorAssembler(inputCols=[col + "_onehot" for col in categorical_cols], outputCol="features")
data_final = assembler.transform(data_encoded).select("features", "label")

# Split data into training and test sets (80% train, 20% test)
train_data, test_data = data_final.randomSplit([0.8, 0.2], seed=42)

# Train a Random Forest Classifier model
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=10)
rf_model = rf.fit(train_data)

# Make predictions on test data
predictions = rf_model.transform(test_data)

# Evaluate model performance
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy:", accuracy)

# rf_model.save("path_to_save_model")



24/04/20 13:38:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/04/20 13:39:37 WARN MemoryStore: Not enough space to cache rdd_214_3 in memory! (computed 3.7 MiB so far)
24/04/20 13:39:37 WARN MemoryStore: Not enough space to cache rdd_214_1 in memory! (computed 20.6 MiB so far)
24/04/20 13:39:37 WARN BlockManager: Persisting block rdd_214_1 to disk instead.
24/04/20 13:39:37 WARN BlockManager: Persisting block rdd_214_3 to disk instead.
24/04/20 13:39:37 WARN MemoryStore: Not enough space to cache rdd_214_2 in memory! (computed 110.4 MiB so far)
24/04/20 13:39:37 WARN MemoryStore: Not enough space to cache rdd_214_4 in memory! (computed 110.4 MiB so far)
24/04/20 13:39:37 WARN BlockManager: Persisting block rdd_214_2 to disk instead.
24/04/20 13:39:37 WARN BlockManager: Persisting block rdd_214_4 to disk instead.
24/04/20 13:39:37 WARN MemoryStore: Not enough space to cache rdd_214_7 in memory! (computed 20.6 MiB so far)
24/04

Accuracy: 0.8230681878202081


In [15]:
# Linear Regression #1 using StringIndexer
#   Predicting Response Times

from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Handle null or NaN values in Response_Time by dropping rows with null labels
df = df.dropna(subset=["Response_Time"])

selected_features = [
    "Call_Type", "City", "Zipcode_of_Incident", "Battalion", "Station_Area",
    "Original_Priority", "Priority", "Final_Priority", "ALS_Unit", "Call_Type_Group",
    "Number_of_Alarms", "Unit_Type", "Unit_sequence_in_call_dispatch",
    "Fire_Prevention_District", "Supervisor_District",
    "Neighborhooods_-_Analysis_Boundaries", "Received_Hour", "Received_DayOfWeek", "Response_Time"
]

data = df.select(selected_features)

# Handle categorical features by indexing and one-hot encoding them
categorical_cols = [
    "Call_Type", "City", "Zipcode_of_Incident", "Battalion", "Station_Area", "Original_Priority",
    "Priority", "Final_Priority", "ALS_Unit", "Call_Type_Group",
    "Unit_Type", "Unit_sequence_in_call_dispatch", "Fire_Prevention_District",
    "Supervisor_District", "Neighborhooods_-_Analysis_Boundaries", "Received_Hour", "Received_DayOfWeek"
]

# Index categorical columns with handling of invalid input set to "keep"
indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep").fit(data)
    for col in categorical_cols
]

# One-hot encode indexed categorical columns
encoders = [
    OneHotEncoder(inputCol=col+"_index", outputCol=col+"_onehot")
    for col in categorical_cols
]

pipeline = Pipeline(stages=indexers + encoders)
data_encoded = pipeline.fit(data).transform(data)

# Prepare data for modeling
assembler = VectorAssembler(inputCols=[col + "_onehot" for col in categorical_cols], outputCol="features")
data_final = assembler.transform(data_encoded).select("features", "Response_Time")

# Split data into training and test sets (80% train, 20% test)
train_data, test_data = data_final.randomSplit([0.8, 0.2], seed=42)

# Train a Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol="Response_Time")
lr_model = lr.fit(train_data)

# Make predictions on test data
predictions = lr_model.transform(test_data)

# Evaluate model performance
evaluator = RegressionEvaluator(labelCol="Response_Time", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

# lr_model.save("path_to_save_model")



24/04/20 13:41:57 WARN Instrumentation: [7ff9d071] regParam is zero, which might cause numerical instability and overfitting.
24/04/20 13:42:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/04/20 13:42:14 WARN Instrumentation: [7ff9d071] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


Root Mean Squared Error (RMSE): 11.61785160992097
R-squared (R2): 0.31256494112208344


In [16]:
# Linear Regression #1 using OneHotEncoder
#   Predicting Response Times


from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Handle null or NaN values in Response_Time by dropping rows with null labels
df = df.dropna(subset=["Response_Time"])

selected_features = [
    "Call_Type", "City", "Zipcode_of_Incident", "Battalion", "Station_Area",
    "Original_Priority", "Priority", "Final_Priority", "ALS_Unit", "Call_Type_Group",
    "Number_of_Alarms", "Unit_Type", "Unit_sequence_in_call_dispatch",
    "Fire_Prevention_District", "Supervisor_District",
    "Neighborhooods_-_Analysis_Boundaries", "Received_Hour", "Received_DayOfWeek", "Response_Time"
]

data = df.select(selected_features)

# Handle categorical features by one-hot encoding them
categorical_cols = [
    "Call_Type", "City", "Zipcode_of_Incident", "Battalion", "Station_Area", "Original_Priority",
    "Priority", "Final_Priority", "ALS_Unit", "Call_Type_Group",
    "Unit_Type", "Unit_sequence_in_call_dispatch", "Fire_Prevention_District",
    "Supervisor_District", "Neighborhooods_-_Analysis_Boundaries", "Received_Hour", "Received_DayOfWeek"
]

# Index string columns with handling of invalid input set to "keep"
indexers = [StringIndexer(inputCol=col, outputCol=col + "_indexed", handleInvalid="keep") for col in categorical_cols]

# One-hot encode indexed categorical columns
encoders = [OneHotEncoder(inputCols=[col + "_indexed"], outputCols=[col + "_onehot"]) for col in categorical_cols]

pipeline = Pipeline(stages=indexers + encoders)
data_encoded = pipeline.fit(data).transform(data)

# Prepare data for modeling
assembler = VectorAssembler(inputCols=[col + "_onehot" for col in categorical_cols], outputCol="features")
data_final = assembler.transform(data_encoded).select("features", "Response_Time")

# Split data into training and test sets (80% train, 20% test)
train_data, test_data = data_final.randomSplit([0.8, 0.2], seed=42)

# Train a Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol="Response_Time")
lr_model = lr.fit(train_data)

# Make predictions on test data
predictions = lr_model.transform(test_data)

# Evaluate model performance
evaluator = RegressionEvaluator(labelCol="Response_Time", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

# lr_model.save("path_to_save_model")

24/04/20 13:44:09 WARN Instrumentation: [09fb8cea] regParam is zero, which might cause numerical instability and overfitting.
24/04/20 13:44:26 WARN Instrumentation: [09fb8cea] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


Root Mean Squared Error (RMSE): 11.61785160992097
R-squared (R2): 0.31256494112208344


In [12]:
# Predicting call volume with linear Regression
#   Predicting Call Volume

from pyspark.sql.functions import col, count
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


# Aggregate and count the call numbers to get the call volumes
call_volumes_df = df.groupBy("Received_DayOfWeek", "Received_Hour").agg(count("Call_Number").alias("Call_Volume"))

selected_features = [
    "Call_Type", "City", "Zipcode_of_Incident", "Battalion", "Station_Area",
    "Original_Priority", "Priority", "Final_Priority", "ALS_Unit", "Call_Type_Group",
    "Number_of_Alarms", "Unit_Type", "Unit_sequence_in_call_dispatch",
    "Fire_Prevention_District", "Supervisor_District",
    "Neighborhooods_-_Analysis_Boundaries", "Received_Hour", "Received_DayOfWeek"
]

data = call_volumes_df.join(df, ["Received_DayOfWeek", "Received_Hour"]).select(selected_features + ["Call_Volume"])

# Handle categorical features by one-hot encoding them
categorical_cols = [
    "Call_Type", "City", "Zipcode_of_Incident", "Battalion", "Station_Area", "Original_Priority",
    "Priority", "Final_Priority", "ALS_Unit", "Call_Type_Group",
    "Unit_Type", "Unit_sequence_in_call_dispatch", "Fire_Prevention_District",
    "Supervisor_District", "Neighborhooods_-_Analysis_Boundaries", "Received_Hour", "Received_DayOfWeek"
]

# Index string columns with handling of invalid input set to "keep"
indexers = [StringIndexer(inputCol=col, outputCol=col + "_indexed", handleInvalid="keep") for col in categorical_cols]

# One-hot encode indexed categorical columns
encoders = [OneHotEncoder(inputCols=[col + "_indexed"], outputCols=[col + "_onehot"]) for col in categorical_cols]

pipeline = Pipeline(stages=indexers + encoders)
data_encoded = pipeline.fit(data).transform(data)

# Prepare data for modeling
assembler = VectorAssembler(inputCols=[col + "_onehot" for col in categorical_cols], outputCol="features")
data_final = assembler.transform(data_encoded).select("features", "Call_Volume")

# Split data into training and test sets (80% train, 20% test)
train_data, test_data = data_final.randomSplit([0.8, 0.2], seed=42)

# Train a Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol="Call_Volume")
lr_model = lr.fit(train_data)

# Make predictions on test data
predictions = lr_model.transform(test_data)

# Evaluate model performance
evaluator = RegressionEvaluator(labelCol="Call_Volume", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print("Root Mean Squared Error (RMSE) for Call Volume Prediction:", rmse)
print("R-squared (R2) for Call Volume Prediction:", r2)

# lr_model.save("path_to_save_model")


24/04/21 13:16:09 WARN Instrumentation: [96e5aa6c] regParam is zero, which might cause numerical instability and overfitting.
24/04/21 13:16:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/04/21 13:16:23 WARN Instrumentation: [96e5aa6c] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


Root Mean Squared Error (RMSE) for Call Volume Prediction: 738.4355418216402
R-squared (R2) for Call Volume Prediction: 0.9499203926619721


In [27]:
spark.stop()